In [5]:
# S-DES Implementation

# Initial permutation for 8-bit data
IP = [2, 6, 3, 1, 4, 8, 5, 7]

# Expansion permutation for 4-bit data to 8 bits
EP = [4, 1, 2, 3, 2, 3, 4, 1]

# Permutation after S-box substitution
P4 = [2, 4, 3, 1]

# Inverse initial permutation for 8-bit data
IP_inverse = [4, 1, 3, 5, 7, 2, 8, 6]

# S-boxes
S0 = [
    [1, 0, 3, 2],
    [3, 2, 1, 0],
    [0, 2, 1, 3],
    [3, 1, 3, 2]
]

S1 = [
    [0, 1, 2, 3],
    [2, 0, 1, 3],
    [3, 0, 1, 0],
    [2, 1, 0, 3]
]

def permute(data, permutation_table):
    binary_str = bin(data)[2:].zfill(len(permutation_table))
    permuted_str = ''.join([binary_str[i - 1] for i in permutation_table])
    return int(permuted_str, 2)

def SDES_encrypt(msg, key):
    # Initial permutation
    msg_permuted = permute(msg, IP)
    
    # Key generation
    key1, key2 = generate_keys(key)
    
    # Round 1
    msg_after_round1 = round_function(msg_permuted, key1)
    
    # Swap
    msg_swapped = ((msg_after_round1 & 0x0F) << 4) | ((msg_after_round1 & 0xF0) >> 4)
    
    # Round 2
    msg_after_round2 = round_function(msg_swapped, key2)
    
    # Inverse initial permutation
    cipher_text = permute(msg_after_round2, IP_inverse)
    
    return cipher_text

def SDES_decrypt(cipher_text, key):
    # Initial permutation
    cipher_permuted = permute(cipher_text, IP)
    
    # Key generation
    key1, key2 = generate_keys(key)
    
    # Round 1
    cipher_after_round1 = round_function(cipher_permuted, key2)
    
    # Swap
    cipher_swapped = ((cipher_after_round1 & 0x0F) << 4) | ((cipher_after_round1 & 0xF0) >> 4)
    
    # Round 2
    cipher_after_round2 = round_function(cipher_swapped, key1)
    
    # Inverse initial permutation
    decrypted_msg = permute(cipher_after_round2, IP_inverse)
    
    return decrypted_msg

def generate_keys(key):
    # 10-bit to 8-bit key
    key = permute(key, [3, 5, 2, 7, 4, 10, 1, 9, 8, 6])
    
    # Split the key into two 5-bit halves
    left_half = key >> 5
    right_half = key & 0x1F
    
    # Circular left shifts
    left_half = ((left_half << 1) | (left_half >> 4)) & 0x1F
    right_half = ((right_half << 1) | (right_half >> 4)) & 0x1F
    
    # Combine halves and permute to get key1
    key1 = permute((left_half << 5) | right_half, P4)
    
    # Circular left shifts
    left_half = ((left_half << 2) | (left_half >> 3)) & 0x1F
    right_half = ((right_half << 2) | (right_half >> 3)) & 0x1F
    
    # Combine halves and permute to get key2
    key2 = permute((left_half << 5) | right_half, P4)
    
    return key1, key2

def round_function(data, key):
    # Expansion permutation
    expanded_data = permute(data, EP)
    
    # XOR with key
    expanded_data ^= key
    
    # S-box substitution
    sbox_output = sbox_substitution(expanded_data)
    
    # Permutation after S-box substitution
    result = permute(sbox_output, P4)
    
    # XOR with the original left half
    result ^= data
    
    return result

def sbox_substitution(data):
    # Split data into 4-bit halves
    left_half = (data >> 4) & 0x0F
    right_half = data & 0x0F
    
    # Apply S-boxes
    sbox_output_left = S0[left_half >> 2][left_half & 0x03]
    sbox_output_right = S1[right_half >> 2][right_half & 0x03]
    
    # Combine S-box outputs
    return (sbox_output_left << 2) | sbox_output_right

# Example Usage:
msg = 0b11011010  # 8-bit message
key = 0b1010000010  # 10-bit key

cipher_text = SDES_encrypt(msg, key)
decrypted_msg = SDES_decrypt(cipher_text, key)

print(f'Message: {bin(msg)}')
print(f'Key: {bin(key)}')
print(f'Cipher Text: {bin(cipher_text)}')
print(f'Decrypted Message: {bin(decrypted_msg)}')
msg == decrypted_msg

Message: 0b11011010
Key: 0b1010000010
Cipher Text: 0b100
Decrypted Message: 0b11011010


True

In [6]:
import random
import csv

MAX_MSG = 256
MAX_KEY = 1024

N_TRAIN_MSG = 64
N_TRAIN_KEY = 256
N_TRAIN = 10000
N_TEST = 20000

# TRAIN
# data = []
# for i in range(N_TRAIN_MSG):
#     msg = i * int(MAX_MSG / N_TRAIN_MSG)
#     for j in range(N_TRAIN_KEY):
#         key = j * int(MAX_KEY / N_TRAIN_KEY)
#         cipher = SDES_encrypt(msg, key)
#         data.append((msg, cipher, key))
# file_name = "SDES_TRAIN.csv"
# delimiter = ','
data = []
for _ in range(N_TRAIN):
    msg = random.randint(0, 255)
    key = random.randint(0, 1024)
    cipher = SDES_encrypt(msg, key)
    
    data.append((msg, cipher, key))
file_name = "SDES_TRAIN.csv"
delimiter = ','

# Write the data to the file
with open(file_name, 'w', newline='') as file:
    writer = csv.writer(file, delimiter=delimiter)
    writer.writerow(["Message", "Ciphertext", "Key"])
    writer.writerows(data)

# TEST
data = []
for _ in range(N_TEST):
    msg = random.randint(0, 255)
    key = random.randint(0, 1024)
    cipher = SDES_encrypt(msg, key)
    
    data.append((msg, cipher, key))
file_name = "SDES_TEST.csv"
delimiter = ','

# Write the data to the file
with open(file_name, 'w', newline='') as file:
    writer = csv.writer(file, delimiter=delimiter)
    writer.writerow(["Message", "Ciphertext", "Key"])
    writer.writerows(data)

NameError: name 'N_T' is not defined